# Preparando o Terreno
- Aqui faremos algumas instalações necessárias para podermos rodar nossos experimentos

In [ ]:
# Ao colocar ! antes de um comando, indicamos que iremos rodar esse comando no terminal da máquina
! pip install wikipedia gradio seaborn cowsay art folium wordcloud
! pip install -U bitsandbytes
! git clone https://github.com/Otavio-Parraga/WorkshoPythonSA.git

import os
os.environ['HF_TOKEN'] = "SEU_TOKEN_HF_AQUI"

# iremos também configurar algumas constantes (apenas para facilitar a vida)
from WorkshoPython import *

# Introdução a Python
- Vamos passar pelas estruturas básicas de Python
- Condicionais, loops, listas, dicionários, funções, etc.

In [ ]:
# Variáveis, tipos básicos e printing

In [ ]:
# formatação e funções de Strings

In [ ]:
# condicionais

In [ ]:
# loops

## Estruturas de Dados

In [ ]:
# estruturas de dados e compreensão

In [ ]:
# indexacao

In [ ]:
# tuplas

In [ ]:
# conjuntos

In [ ]:
# dicionarios

## Funções

In [ ]:
# Funções


In [ ]:
# retornos em tuplas

In [ ]:
# *args

In [ ]:
# **kwargs

In [ ]:
# type

In [ ]:
# help

In [ ]:
# map

In [ ]:
# filter

In [ ]:
# zip

In [ ]:
# enumerate

In [ ]:
# with

# Algumas bibliotecas interessantes

In [ ]:
# Temos alguns easter eggs na linguagem


In [ ]:
# Podemos deixar as iterações bonitas


In [ ]:
# Um pouquinho de brincadeira não faz mal


In [ ]:
from art import tprint

tprint("Arte!")

In [ ]:
import folium

# Cria um mapa centrado no ponto indicado
# As coordenadas são [Latitude, Longitude]
mapa = folium.Map(location=[-30.05925097055111, -51.172985771163894], zoom_start=15)

# Adiciona um "pin" (marcador)
folium.Marker(
    location=[-30.05925097055111, -51.172985771163894],
    popup="PUCRS",
    tooltip="Clique aqui!"
).add_to(mapa)

# Apenas chame a variável 'mapa' na última linha da célula
mapa

## Para Ciência de Dados

In [ ]:
import pandas as pd

df = pd.read_csv(DATA_LINK)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

descriptions = df['description'].str.cat(sep=' ')
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(descriptions)
plt.imshow(wordcloud)

In [ ]:
# E podemos pesquisar na Wikipedia
import wikipedia
wikipedia.set_lang("pt")

def busca_wikipedia(termo: str, sentences: int = 2) -> str:
    try:
        resumo = wikipedia.summary(termo, sentences=sentences)
        return resumo
    except:
        return "Não encontrei nada sobre isso na Wikipedia."

print(busca_wikipedia("John Mayer", 20))

# Podemos integrar com LLMs facilmente

In [ ]:
from transformers import pipeline

model = pipeline("text-generation",
                 model=QWEN_PATH,
                 device_map="auto",
                 model_kwargs={"quantization_config": QUANT_CONFIG})

In [ ]:
import gradio as gr


def chat_with_llm(message, history):
    """
    Recebe a mensagem do usuário e o histórico do chat,
    envia para o modelo e retorna a resposta.
    """
    messages = [{"role": "system", "content": "Você é um assistente útil e educado."}] + history

    # Adiciona a nova mensagem
    messages.append({"role": "user", "content": message})

    completion = model(messages,
                       max_new_tokens=512,
                       return_full_text =False,
                       eos_token_id=model.tokenizer.eos_token_id,
                       temperature=0.7,
                       top_p=0.9)[0]['generated_text']
    return completion


# --- Interface com Gradio ---
demo = gr.ChatInterface(
    fn=chat_with_llm,
    type="messages",
    title="💬 Chatbot via Hugging Face",
    description="Workshop de Python",
    # theme="soft",
    # examples=["Explique o que é aprendizado de máquina.", "Resuma a história do Python."],
)

# --- Inicia o app ---
demo.launch(debug=True)